## Depêndencias

In [1]:
#!pip install torch
#!pip install transformers
#pip install dotenv

In [12]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, Gemma3ForCausalLM
from dotenv import load_dotenv
from huggingface_hub import login
import os


load_dotenv()

True

In [3]:
hf_token = os.getenv("HF_TOKEN")
login(token=hf_token)
print("Login realizado com sucesso!")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Login realizado com sucesso!


## Instanciando modelos e tokenizadores

In [4]:
tokenizer_base = AutoTokenizer.from_pretrained(
    "google/gemma-3-1b-pt",
)

model_base = AutoModelForCausalLM.from_pretrained(
    "google/gemma-3-1b-pt",
    dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="sdpa"
)


## Testando modelo Base

In [9]:
prompt = "hi, how are you?"

In [10]:
input_ids = tokenizer_base(prompt, return_tensors="pt").to(model_base.device)
output = model_base.generate(**input_ids, max_new_tokens=256)
print(tokenizer_base.decode(output[0], skip_special_tokens=True))

hi, how are you? I would be happy if you could help me with a small question.

I have the following environment

OS: Debian 9

VFS: vfs tools 2.40.0

kernel: linux-6.3.0-23-amd64

python: 3.7.0

Here's the result of my <code>ldd</code> :

<code>libm.so.6 => /lib/i386-linux-gnu/libm.so.6 (0x00007ffff7702000)
libdl.so.2 => /lib/i386-linux-gnu/libdl.so.2 (0x00007ffff7701000)
libdl-tech.so.2 => /lib/i386-linux-gnu/libdl-tech.so.2 (0x00007ffff7700000)
libresolv.so.2 => /lib/i386-linux-gnu/libresolv.so.2 (0x00007ffff77000


In [11]:
prompt = "write a poem"

input_ids = tokenizer_base(prompt, return_tensors="pt").to(model_base.device)
output = model_base.generate(**input_ids, max_new_tokens=256)
print(tokenizer_base.decode(output[0], skip_special_tokens=True))

write a poem using a number lineI will need your number line and a number.  

Write 6 number sentences about the number 3.

Write 2 number sentences about the number 8.

Write 6 number sentences about the number 12.

Write 6 number sentences about the number 24.

Write 2 number sentences about the number 54.

Write 6 number sentences about the number 108.

Write 2 number sentences about the number 216.

Write 6 number sentences about the number 400.

Write 2 number sentences about the number 800.

Write 6 number sentences about the number 1600.

Write 2 number sentences about the number 3200.

Write 4 number sentences about the number 18000.

Write 4 number sentences about the number 36000.

Write 2 number sentences about the number 84000.

Write 4 number sentences about the number 360000.

Write 2 number sentences about the number 880000.


Nesse ponto é possível perceber que o modelo "base" não é capaz de seguir instruções simples que lhe foram solicitadas.

## Testando modelo Instruct

In [19]:
model_id = "google/gemma-3-1b-it"

model = Gemma3ForCausalLM.from_pretrained(
    model_id,
    dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="sdpa"
).eval()

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [20]:
prompt = "hi, how are you?"

In [21]:
messages = [
    {"role": "system", "content": [{"type": "text", "text": "You are a helpful assistant."}]},
    {"role": "user",   "content": [{"type": "text", "text": prompt}]},
]

chat_text = tokenizer.apply_chat_template(
    [messages],                
    add_generation_prompt=True,
    tokenize=False         
)

inputs = tokenizer(chat_text, return_tensors="pt").to(model.device)

In [23]:
out_ids = model.generate(**inputs, max_new_tokens=128, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(out_ids[0], skip_special_tokens=True))

user
You are a helpful assistant.

hi, how are you?
model
I’m doing well, thank you for asking! As a large language model, I don’t experience feelings in the same way humans do, but I’m functioning perfectly and ready to help you with whatever you need. 😊 How are *you* doing today? 

Do you want to talk about something, or perhaps need some assistance with a task?


In [24]:
prompt = "write a poem"

In [25]:
messages = [
    {"role": "system", "content": [{"type": "text", "text": "You are a helpful assistant."}]},
    {"role": "user",   "content": [{"type": "text", "text": prompt}]},
]

chat_text = tokenizer.apply_chat_template(
    [messages],                
    add_generation_prompt=True,
    tokenize=False         
)

inputs = tokenizer(chat_text, return_tensors="pt").to(model.device)

In [26]:
out_ids = model.generate(**inputs, max_new_tokens=128, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(out_ids[0], skip_special_tokens=True))

user
You are a helpful assistant.

write a poem
model
Okay, here’s a poem for you:

**Golden Hour's Whisper**

The sun descends, a honeyed hue,
Painting clouds in shades of gold and true.
The air grows still, a gentle grace,
As shadows lengthen, finding their place.

A robin sings a quiet plea,
A melody of tranquility, free.
The leaves above, a fiery sheen,
Reflecting light, a vibrant scene.

The world slows down, a peaceful sigh,
As day surrenders to the evening sky.
A moment held, a cherished view,
Of golden hours, serene and new


é possível perceber a nítida diferença de comportamento entre os dois modelos!

## Testando modelo base nas perguntas do InstructGPT

In [27]:
prompt = "Explain the theory of gravity to a 6 year old."

input_ids = tokenizer_base(prompt, return_tensors="pt").to(model_base.device)
output = model_base.generate(**input_ids, max_new_tokens=256)
print(tokenizer_base.decode(output[0], skip_special_tokens=True))

Explain the theory of gravity to a 6 year old.

Explain what a photon is.

Explain the theory of relativity.

Explain why it’s impossible to travel faster than light.

Explain why light always travels in a straight line.

Explain what time dilation is and when it happens.

Explain why people can’t reach the moon.

Explain why you can’t see a black hole.

Explain what a black hole is and how it happens.

Explain why a black hole doesn’t have a blackness.

Explain how a black hole can be black.

Explain how light and other objects can be made to move faster than light.

Explain why it is impossible to go faster than light.

Explain why it is impossible to fall faster than light.

Explain why it is impossible to accelerate faster than light.

Explain why it is impossible to create a black hole.


In [28]:
prompt = "Explain the theory of relativity to a 6 year old in a few sentences."

input_ids = tokenizer_base(prompt, return_tensors="pt").to(model_base.device)
output = model_base.generate(**input_ids, max_new_tokens=256)
print(tokenizer_base.decode(output[0], skip_special_tokens=True))

Explain the theory of relativity to a 6 year old in a few sentences. I am a physics graduate and have never heard of this. Thank you

I would use what I did for my 7 year old daughter as an example. She asked my wife "is Earth flat?" - which is not a scientific question, but an example of how she may use scientific ideas to create an imaginary world where what she thinks of is true.


<blockquote><b>20:33:43</b> [300]
To the rest of you who are not geeks, Earth is not flat. It has an axis, and it is tilted at a few degrees. It is rotating and orbiting the sun. The sun is one of millions of stars in our galaxy and in our universe. The milky way is 100,000 light years across. The nearest other planet to the sun is Neptune (which is a gas giant). We are close enough to the Sun to have a year that is 365 days, just like any other year (though it takes more time to rotate). Our year is not like any other in the universe, because we are moving around in space.
<b>20:34:53</b> [300]
As for wh

In [31]:
prompt = "Explain the big bang theory to a 6 year old."

input_ids = tokenizer_base(prompt, return_tensors="pt").to(model_base.device)
output = model_base.generate(**input_ids, max_new_tokens=256)
print(tokenizer_base.decode(output[0], skip_special_tokens=True))

Explain the big bang theory to a 6 year old. Explain the dark ages in the history of the world to a 10 year old. Explain the atomic bomb to a 12 year old. Explain the life of Albert Einstein to a 13 year old. Explain why it is called the theory of gravity to a 15 year old. Explain why our world is not made of water to a 16 year old. And explain the universe as it is today to a 18 year old.

The main parts of the theory of relativity is the idea that time is relative to an observer. Therefore the time that an observer feels for himself is relative to his surroundings. Because, if one looks at an object, from the position of another object at a different time, then the relative time that person feels is different than when they observe the object at the same time. If you look at an object from an airplane, it seems to slow down relative to the same object at ground level. A clock that ticks slower than a clock on a person on the ground will seem like a clock ticks faster than a clock on 

In [32]:
prompt = "Explain evolution to a 6 year old."

input_ids = tokenizer_base(prompt, return_tensors="pt").to(model_base.device)
output = model_base.generate(**input_ids, max_new_tokens=256)
print(tokenizer_base.decode(output[0], skip_special_tokens=True))

Explain evolution to a 6 year old. That's a tough question isn't it? I find that very difficult too, but I have been doing it for a while. I must tell you that I am a very good evolutionist. Well not <em>that</em> good, I am not <em>that</em> good. I have a few good moments here and there, but I don't think I can claim to be an expert. But I have always been able to explain evolution to some extent.

I must admit that I had a difficult time with this lesson. For one thing, I didn't like the lesson. As I said in my introduction I don't particularly like evolution, and the way the lesson was presented (through science) made me quite nervous. That and the fact that there was some kind of math involved. Math is not my forte, but I have to do math.

For this lesson we are to tell a 6 year old about evolution. I was told that I was to use the Internet for that. But I just wasn't sure how to do it. The problem with the Internet is that you don't always get any explanation as to how it works, 

In [33]:
prompt = "Why is it important to eat socks after meditating?"

input_ids = tokenizer_base(prompt, return_tensors="pt").to(model_base.device)
output = model_base.generate(**input_ids, max_new_tokens=256)
print(tokenizer_base.decode(output[0], skip_special_tokens=True))

Why is it important to eat socks after meditating?

"Before I meditate, I wear socks."

"What happens when I meditate and my socks fall off?"

"Why is it important to eat socks after meditating? Or should I say, why is it important to eat <em>other things</em> after meditating?"

"Should I eat after meditating to clean my mind?"

These are just some of the many questions I receive when I teach about meditation. I love teaching about it, but sometimes it makes me wonder <em>why</em> I teach it.

I teach about meditation, and I ask questions, because I have spent countless years exploring it, thinking about it, talking about it, writing about it, practicing it, debating about it, sharing it, and discussing it. It is a passion of mine and it deeply connects me to the universe. It has given me insight into life that I wish I had the time to experience for myself and others to share.

My question is not about the benefits of meditating that I have observed, but about the connection that I h

In [34]:
prompt = "What happens if you fire a cannonball directly at a pumpkin at high speeds?"

input_ids = tokenizer_base(prompt, return_tensors="pt").to(model_base.device)
output = model_base.generate(**input_ids, max_new_tokens=256)
print(tokenizer_base.decode(output[0], skip_special_tokens=True))

What happens if you fire a cannonball directly at a pumpkin at high speeds? How does its shape change? How does its motion change? What kinds of things can happen to pumpkins when you fire a cannonball at them?

In this fun activity, you’ll learn how to use an equation that will allow you to calculate an unknown velocity and an unknown force and then use it to calculate an unknown displacement.

We start with the physics of projectile motion by looking at a cannonball fired directly at a pumpkin. We first measure the velocity of the pumpkin and the velocity of the cannonball and use those numbers to determine what we know and what we don’t know.

Then we calculate an unknown speed and an unknown force and then use the formula to find what we don’t know. Finally, we test the equation by firing a cannonball straight at a pumpkin and recording the displacement from the starting point.

This activity can be used in any physics or science class at any grade level. It is also a great way to 

Por mais que o texto vindo dos prompts acima tenha alguma relação com o tema do prompt, podemos ver claramente a incapacidade de seguir instruções, ou seja, fornecer respostas condizentes com a pergunta especificada.